<a href="https://colab.research.google.com/github/ChiliJoe/genai-intro/blob/main/ChatGPT_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an expansion of tutorial from https://youtu.be/0-zlUy7VUjg using real-world API from openweathermap.org.

Let's use the openweathermap current weather API that requires geo code as input.

In [12]:
%%capture
# Set openweathermap.org API key
%env OWM_API_KEY=

# Set OpenAI API key
%env OPENAI_API_KEY=

In [13]:
%%capture
!rm -rf helper
!git clone https://github.com/ChiliJoe/genai-intro.git helper

from helper.weather import function_list

# Vanilla OpenAI

In [14]:
%%capture
!pip install openai

In [15]:
from IPython.display import display, Markdown
from helper.demolib import Color

import openai
import os
import json

openai.api_key  = os.getenv('OPENAI_API_KEY')

class Chat:
  def __init__(self, system_message, function_list):
    self.messages = [{"role": "system", "content": system_message}]
    self.function_list = function_list
    self.function_descriptions = [f["description"] for f in function_list.values()]

  def ask(self, role_content, role="user", function_name=None):
    if role == "function":
      self.messages.append({"role": role, "name": function_name, "content": role_content})
    else:
      self.messages.append({"role": role, "content": role_content})

    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=self.messages,
        functions=self.function_descriptions,
        function_call="auto"
    )
    ai_message = res.choices[0].message
    self.messages.append(ai_message)

    # process function call
    if ai_message["content"] is None and ai_message["function_call"]:
      target_function_name = ai_message["function_call"]["name"]
      target_function = self.function_list[target_function_name]["function"]
      func_params = json.loads(ai_message['function_call']['arguments'])

      print(Color.cyan(f"Invoking {target_function_name} with parameters {func_params}"))
      function_response = target_function(**func_params)
      print(Color.white("Function response: "), end="")
      print(Color.magenta(f"{function_response}"))

      self.ask(role_content=json.dumps(function_response),
               role="function",
               function_name=target_function_name
               )

    elif ai_message["content"]:
      print(Color.green("===== Final response: ====="))
      display(Markdown(ai_message["content"]))

In [30]:
sysmessage="""
You are a helpful AI chatbot that responds in conversational manner.
"""
chat = Chat(sysmessage, function_list)

In [31]:
chat.ask("Where does the head of US government live?")

===== Final response: =====


The head of the US government, which is the President, lives in the White House in Washington, D.C.

In [32]:
chat.ask("What is the weather like there?")

Invoking get_location_data with parameters {'city': 'Washington', 'country': 'US'}
Function response: [{'name': 'Washington', 'local_names': {'sg': 'Washington D.C.', 'mn': 'Вашингтон хот', 'zu': 'Washington, D.C.', 'zh': '华盛顿 / 蓽盛頓', 'sw': 'Washington, D.C.', 'vi': 'Hoa Thịnh Đốn', 'cy': 'Washington, D.C.', 'fy': 'Washington, D.C.', 'sh': 'Washington, D.C.', 'cs': 'Washington, D.C.', 'is': 'Washington', 'sq': 'Uashington D.C.', 'ks': 'واشنگٹن', 'mk': 'Вашингтон', 'sn': 'Washington, DC', 'ms': 'Washington, D.C.', 'no': 'Washington, D.C.', 'mr': 'वॉशिंग्टन, डी.सी.', 'gd': 'Washington, D.C.', 'pl': 'Waszyngton', 'ru': 'Вашингтон', 'te': 'వాషింగ్టన్, డి.సి.', 'ko': '워싱턴', 'fr': 'Washington', 'kw': 'Washington, D.C.', 'be': 'Вашынгтон', 'vo': 'Washington', 'hu': 'Washington', 'sm': 'Uasigitone D.C.', 'af': 'Washington, D.C.', 'fa': 'واشنگتن', 'gr': 'Οὐασιντώνη', 'et': 'Washington', 'lv': 'Vašingtona', 'tw': 'Wɔhyeten', 'yi': 'וואשינגטאן די סי', 'ug': 'ۋاشىنگتون', 'ja': 'ワシントン', 'ee': 'Wash

The current weather in Washington, D.C. is overcast clouds with a temperature of 30.18°C. It feels like 35.48°C with a humidity of 70%. The minimum temperature is 27.74°C and the maximum temperature is 32.51°C. The wind speed is 5.66 m/s with a direction of 170 degrees.